In [26]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint
import data
from tqdm import tnrange, tqdm_notebook, trange

In [2]:
vocab_size = 1900
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'
print(len(d))
x, y = d[10:20], d[11:21]
print([np.argmax(j) for j in x])
print([np.argmax(j) for j in y])

['in', 'the', 'beginning', 'god', 'created', 'the', 'heaven', 'and', 'the', 'earth', '.', 'and', 'the', 'earth', 'was', 'without', 'form', ',', 'and', 'void', ';', 'and', 'darkness', 'was', 'upon', 'the', 'face', 'of', 'the', 'deep', '.', 'and', 'the', 'spirit', 'of', 'god', 'moved', 'upon', 'the', 'face']
1909
29602
[5, 1, 3, 45, 24, 451, 1816, 2, 1, 1698]
[1, 3, 45, 24, 451, 1816, 2, 1, 1698, 9]


In [3]:
# hyper-parameters
input_size = output_size = vocab_size
hidden_layer = 300
inp_out_size = vocab_size
learning_rate = 0.1
num_steps = 20

In [4]:
tf.reset_default_graph()
initializer = tf.random_normal_initializer(mean=0, stddev=0.001, dtype=tf.float32)
Wxh = tf.get_variable('Wxh', shape=[input_size, hidden_layer], initializer=initializer)
Whh = tf.get_variable('Whh', shape=[hidden_layer, hidden_layer], initializer=initializer)
Why = tf.get_variable('Why',shape=[hidden_layer, output_size], initializer=initializer)
by = tf.get_variable('by', shape=[output_size], initializer=initializer)
# weights associated with update gate
Wxz = tf.get_variable('Wxz', shape=[input_size, hidden_layer], initializer=initializer)
Whz = tf.get_variable('Whz', shape=[hidden_layer, hidden_layer], initializer=initializer)
# weights associated with the reset gate
Wxr = tf.get_variable('Wxr', shape=[input_size, hidden_layer], initializer=initializer)
Whr = tf.get_variable('Whr', shape=[hidden_layer, hidden_layer], initializer=initializer)

In [5]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
    z = tf.nn.sigmoid(tf.matmul(i, Wxz) + tf.matmul(p, Whz))    # update gate
    r = tf.nn.sigmoid(tf.matmul(i, Wxr) + tf.matmul(p, Whr))    # reset gate
    h_ = tf.nn.tanh(tf.matmul(i, Wxh) + tf.matmul(tf.mul(p, r), Whh))
    h = tf.nn.tanh(tf.mul(tf.sub(tf.ones_like(z), z), h_) + tf.mul(z, p))
    return tf.reshape(h, [hidden_layer])

In [6]:
a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
initial = tf.placeholder(shape=[hidden_layer], dtype=tf.float32)
states = tf.nn.dropout(tf.scan(recurrence, a, initializer=initial), keep_prob=0.8)
outputs = tf.nn.softmax(tf.matmul(states, Why) + by)
loss = -tf.reduce_sum(b*tf.log(outputs))
# loss = tf.sqrt(tf.reduce_sum(tf.square(tf.sub(outputs, b))))
optimizer = tf.train.AdagradOptimizer(learning_rate)

# clipping gradients between -1 and 1.
grad_var_pairs = optimizer.compute_gradients(loss, tf.trainable_variables())
clipped_grad_var_pairs = [(tf.clip_by_value(gv[0], -4, 4), gv[1]) for gv in grad_var_pairs]
optimize_op = optimizer.apply_gradients(clipped_grad_var_pairs)

# optimize_op = optimizer.minimize(loss)

In [7]:
def generate(sess, n):
    x, _ = data.sample(d, 1)
    gen = [id_to_token[np.argmax(x[0])]]
    h = np.zeros(hidden_layer)
    for i in range(n):
        o, h = sess.run([outputs, states], {a:x, initial: h})
        h = h.reshape(hidden_layer)
        o = np.argmax(o[0])
        gen.append(id_to_token[o])
        x = [0] * inp_out_size
        x[o] = 1
#         print np.argmax(x)
        x = [x]
    print(' '.join(gen))

In [20]:
init = tf.initialize_all_variables()
saver = tf.train.Saver()
sess = tf.Session()
ix = 0

In [21]:
# if already trained previously, just restore
to_restore = False

if to_restore:
    saver.restore(sess, 'model.ckpt')
else:
    sess.run(init)

In [ ]:
iterations = 4000
h = np.zeros(hidden_layer)
t = trange(iterations, miniters=100)
for i in t:
#     x, y = data.sample(d, num_steps)
    if ix + num_steps >= len(d):
        ix = 0
#         t.set_description('one epoch complete')
    h = np.zeros(hidden_layer)
    x, y = d[ix : ix + num_steps], d[ix + 1 : ix + num_steps + 1]   
    l, o, _ = sess.run([loss, outputs, optimize_op], {a: x, b: y, initial: h})
#     if i % 50 == 0:
    t.set_description('loss %i' %l)
#         generate(sess, 50)
    ix += num_steps


loss 64:   8%|▊         | 300/4000 [00:45<09:24,  6.55it/s]

In [ ]:
# saving model
# saver.save(sess, 'model.ckpt')

In [14]:
generate(sess, 100)

much traction which itself required marketing dollars.we were stuck in a vicious loop , to be able to pursue his robotics and robotics passion before that time and that should be part of your product market fit , an industry has , by a month ago , this is an one of the human computer interaction ) . image is very a time . we have a great set to new yourself focused and technology innovation . in the very half of our product or on . and robotics and we could do any path we wanted to and that helped


In [ ]:
# sess.run(Whh)